In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import datetime

In [2]:
# masterMLpublic = pd.read_csv("/kaggle/input/mma-differentials-and-elo/masterMLpublic.csv")
# masterMLpublic

In [3]:
# for col in masterMLpublic.columns:
#     print(col)

In [4]:
masterdataframe = pd.read_csv("/kaggle/input/mma-differentials-and-elo/masterdataframe.csv")
masterdataframe

,date,fight_url,event_url,result,fighter,opponent,division,stance,dob,method,...,recent_avg_clinch_strikes_attempts_per_min,precomp_recent_avg_clinch_strikes_attempts_per_min,avg_ground_strikes_landed_per_min,precomp_avg_ground_strikes_landed_per_min,recent_avg_ground_strikes_landed_per_min,precomp_recent_avg_ground_strikes_landed_per_min,avg_ground_strikes_attempts_per_min,precomp_avg_ground_strikes_attempts_per_min,recent_avg_ground_strikes_attempts_per_min,precomp_recent_avg_ground_strikes_attempts_per_min
0,1994-03-11,http://ufcstats.com/fight-details/4acab67848e7...,http://ufcstats.com/event-details/a6a9ab5a824e...,0,Sean Daugherty,Scott Morris,Open Weight,NaN,1975-12-04,SUB,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
1,1994-03-11,http://ufcstats.com/fight-details/4acab67848e7...,http://ufcstats.com/event-details/a6a9ab5a824e...,1,Scott Morris,Sean Daugherty,Open Weight,Orthodox,NaN,SUB,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
2,1994-03-11,http://ufcstats.com/fight-details/4b9ae533ccb3...,http://ufcstats.com/event-details/a6a9ab5a824e...,0,Ray Wizard,Patrick Smith,Open Weight,NaN,NaN,SUB,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
3,1994-03-11,http://ufcstats.com/fight-details/4b9ae533ccb3...,http://ufcstats.com/event-details/a6a9ab5a824e...,1,Patrick Smith,Ray Wizard,Open Weight,Orthodox,1963-08-28,SUB,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
4,1994-03-11,http://ufcstats.com/fight-details/ccee020be2e8...,http://ufcstats.com/event-details/a6a9ab5a824e...,0,David Levicki,Johnny Rhodes,Open Weight,NaN,NaN,KO/TKO,...,NaN,NaN,0.081855,NaN,NaN,NaN,0.081855,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13317,2022-06-25,http://ufcstats.com/fight-details/c57c8e22a3e8...,http://ufcstats.com/event-details/eb42d4febfaf...,1,Josh Parisian,Alan Baudot,Heavyweight,Orthodox,1989-06-28,KO/TKO,...,1.178248,1.812958,0.557851,0.000000,0.743802,0.000000,0.774793,0.000000,1.033058,0.000000
13318,2022-06-25,http://ufcstats.com/fight-details/3c98739eb42f...,http://ufcstats.com/event-details/eb42d4febfaf...,0,Neil Magny,Shavkat Rakhmonov,Welterweight,Orthodox,1987-08-03,SUB,...,0.533333,0.600000,0.995168,1.021866,0.167001,0.066667,1.304373,1.331387,0.267336,0.066667
13319,2022-06-25,http://ufcstats.com/fight-details/3c98739eb42f...,http://ufcstats.com/event-details/eb42d4febfaf...,1,Shavkat Rakhmonov,Neil Magny,Welterweight,Orthodox,1994-10-23,SUB,...,0.186047,0.614618,0.835357,0.645581,1.113809,0.645581,1.515895,1.151628,2.021193,1.151628
13320,2022-06-25,http://ufcstats.com/fight-details/1f5f59924b59...,http://ufcstats.com/event-details/eb42d4febfaf...,0,Arman Tsarukyan,Mateusz Gamrot,Lightweight,Orthodox,1996-10-11,U-DEC,...,0.080000,0.177778,2.190916,2.529402,4.378803,4.458803,2.772149,3.194174,5.446126,5.566126


In [5]:
# for col in masterdataframe.columns:
#     print(col)

In [6]:
masterdataframe["division"].value_counts()

division
Lightweight              2362
Welterweight             2324
Middleweight             1794
Heavyweight              1266
Featherweight            1252
Light Heavyweight        1222
Bantamweight             1098
Flyweight                 524
Women's Strawweight       470
Women's Bantamweight      338
Women's Flyweight         330
Open Weight               204
Catch Weight               90
Women's Featherweight      46
Super Heavyweight           2
Name: count, dtype: int64

In [7]:
lightweight = masterdataframe[masterdataframe["division"] == "Lightweight"]
lightweight

,date,fight_url,event_url,result,fighter,opponent,division,stance,dob,method,...,recent_avg_clinch_strikes_attempts_per_min,precomp_recent_avg_clinch_strikes_attempts_per_min,avg_ground_strikes_landed_per_min,precomp_avg_ground_strikes_landed_per_min,recent_avg_ground_strikes_landed_per_min,precomp_recent_avg_ground_strikes_landed_per_min,avg_ground_strikes_attempts_per_min,precomp_avg_ground_strikes_attempts_per_min,recent_avg_ground_strikes_attempts_per_min,precomp_recent_avg_ground_strikes_attempts_per_min
198,1997-02-07,http://ufcstats.com/fight-details/18af785da73b...,http://ufcstats.com/event-details/96eff1a628ad...,0,Rainy Martinez,Jerry Bohlander,Lightweight,Orthodox,NaN,SUB,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
199,1997-02-07,http://ufcstats.com/fight-details/18af785da73b...,http://ufcstats.com/event-details/96eff1a628ad...,1,Jerry Bohlander,Rainy Martinez,Lightweight,Orthodox,1974-02-12,SUB,...,0.111111,0.184776,0.272942,0.363922,0.327090,0.363922,0.623495,0.831327,0.794495,0.831327
200,1997-02-07,http://ufcstats.com/fight-details/e45feb9866eb...,http://ufcstats.com/event-details/96eff1a628ad...,0,Wallid Ismail,Yoshiki Takahashi,Lightweight,Orthodox,1968-02-23,U-DEC,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
201,1997-02-07,http://ufcstats.com/fight-details/e45feb9866eb...,http://ufcstats.com/event-details/96eff1a628ad...,1,Yoshiki Takahashi,Wallid Ismail,Lightweight,Southpaw,1969-03-13,U-DEC,...,NaN,NaN,0.533333,NaN,NaN,NaN,0.666667,NaN,NaN,NaN
206,1997-02-07,http://ufcstats.com/fight-details/2963e9a4b8c1...,http://ufcstats.com/event-details/96eff1a628ad...,0,Nick Sanzo,Jerry Bohlander,Lightweight,NaN,NaN,SUB,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13291,2022-06-18,http://ufcstats.com/fight-details/4e6218b6b892...,http://ufcstats.com/event-details/a0a680fe2f6c...,1,Damir Ismagulov,Guram Kutateladze,Lightweight,Orthodox,1991-02-03,S-DEC,...,0.244444,0.066667,0.546667,0.683333,0.666667,0.666667,0.786667,0.983333,0.866667,0.866667
13314,2022-06-25,http://ufcstats.com/fight-details/029a12d36a46...,http://ufcstats.com/event-details/eb42d4febfaf...,0,Christos Giagos,Thiago Moises,Lightweight,Orthodox,1990-01-23,SUB,...,0.322170,0.344392,0.674544,0.741999,0.055710,0.344599,1.278332,1.406165,0.055710,0.411266
13315,2022-06-25,http://ufcstats.com/fight-details/029a12d36a46...,http://ufcstats.com/event-details/eb42d4febfaf...,1,Thiago Moises,Christos Giagos,Lightweight,Orthodox,1995-03-23,SUB,...,0.164819,0.056711,0.235931,0.265422,0.018904,0.018904,0.301491,0.339178,0.037807,0.037807
13320,2022-06-25,http://ufcstats.com/fight-details/1f5f59924b59...,http://ufcstats.com/event-details/eb42d4febfaf...,0,Arman Tsarukyan,Mateusz Gamrot,Lightweight,Orthodox,1996-10-11,U-DEC,...,0.080000,0.177778,2.190916,2.529402,4.378803,4.458803,2.772149,3.194174,5.446126,5.566126


In [8]:
lightweight["fighter"].nunique()

500

In [9]:
# Convert the date column from object into datetime
# lightweight["date"] = lightweight["date"].astype("datetime64[ns]")
lightweight["date"] = pd.to_datetime(lightweight["date"]).dt.date

In [10]:
date_threshold = datetime.date(2021, 1, 1)
early_lightweight = lightweight[lightweight["date"] < date_threshold]
late_lightweight = lightweight[lightweight["date"] >= date_threshold]

In [11]:
early_lightweight

,date,fight_url,event_url,result,fighter,opponent,division,stance,dob,method,...,recent_avg_clinch_strikes_attempts_per_min,precomp_recent_avg_clinch_strikes_attempts_per_min,avg_ground_strikes_landed_per_min,precomp_avg_ground_strikes_landed_per_min,recent_avg_ground_strikes_landed_per_min,precomp_recent_avg_ground_strikes_landed_per_min,avg_ground_strikes_attempts_per_min,precomp_avg_ground_strikes_attempts_per_min,recent_avg_ground_strikes_attempts_per_min,precomp_recent_avg_ground_strikes_attempts_per_min
198,1997-02-07,http://ufcstats.com/fight-details/18af785da73b...,http://ufcstats.com/event-details/96eff1a628ad...,0,Rainy Martinez,Jerry Bohlander,Lightweight,Orthodox,NaN,SUB,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
199,1997-02-07,http://ufcstats.com/fight-details/18af785da73b...,http://ufcstats.com/event-details/96eff1a628ad...,1,Jerry Bohlander,Rainy Martinez,Lightweight,Orthodox,1974-02-12,SUB,...,0.111111,0.184776,0.272942,0.363922,0.327090,0.363922,0.623495,0.831327,0.794495,0.831327
200,1997-02-07,http://ufcstats.com/fight-details/e45feb9866eb...,http://ufcstats.com/event-details/96eff1a628ad...,0,Wallid Ismail,Yoshiki Takahashi,Lightweight,Orthodox,1968-02-23,U-DEC,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
201,1997-02-07,http://ufcstats.com/fight-details/e45feb9866eb...,http://ufcstats.com/event-details/96eff1a628ad...,1,Yoshiki Takahashi,Wallid Ismail,Lightweight,Southpaw,1969-03-13,U-DEC,...,NaN,NaN,0.533333,NaN,NaN,NaN,0.666667,NaN,NaN,NaN
206,1997-02-07,http://ufcstats.com/fight-details/2963e9a4b8c1...,http://ufcstats.com/event-details/96eff1a628ad...,0,Nick Sanzo,Jerry Bohlander,Lightweight,NaN,NaN,SUB,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11743,2020-12-05,http://ufcstats.com/fight-details/46860358da8b...,http://ufcstats.com/event-details/398b440e73d5...,1,Gabriel Benitez,Justin Jaynes,Lightweight,Southpaw,1988-06-15,KO/TKO,...,0.000000,0.000000,1.329221,1.368511,0.325203,3.589744,2.319144,2.441326,0.406504,6.666667
11754,2020-12-12,http://ufcstats.com/fight-details/cb7616f92a9a...,http://ufcstats.com/event-details/d0d43cb9b14f...,0,Renato Moicano,Rafael Fiziev,Lightweight,Orthodox,1989-05-21,KO/TKO,...,0.163265,0.000000,0.209804,0.233115,0.000000,0.000000,0.339216,0.376906,0.000000,0.000000
11755,2020-12-12,http://ufcstats.com/fight-details/cb7616f92a9a...,http://ufcstats.com/event-details/d0d43cb9b14f...,1,Rafael Fiziev,Renato Moicano,Lightweight,Switch,1993-03-05,KO/TKO,...,0.755556,0.755556,0.094558,0.044444,0.126077,0.044444,0.094558,0.044444,0.126077,0.044444
11764,2020-12-12,http://ufcstats.com/fight-details/b7b867eb3c3c...,http://ufcstats.com/event-details/d0d43cb9b14f...,0,Tony Ferguson,Charles Oliveira,Lightweight,Orthodox,1984-02-12,U-DEC,...,0.088889,0.300000,0.442409,0.456668,0.066667,0.133333,0.607901,0.631895,0.066667,0.200000


In [12]:
late_lightweight

,date,fight_url,event_url,result,fighter,opponent,division,stance,dob,method,...,recent_avg_clinch_strikes_attempts_per_min,precomp_recent_avg_clinch_strikes_attempts_per_min,avg_ground_strikes_landed_per_min,precomp_avg_ground_strikes_landed_per_min,recent_avg_ground_strikes_landed_per_min,precomp_recent_avg_ground_strikes_landed_per_min,avg_ground_strikes_attempts_per_min,precomp_avg_ground_strikes_attempts_per_min,recent_avg_ground_strikes_attempts_per_min,precomp_recent_avg_ground_strikes_attempts_per_min
11816,2021-01-20,http://ufcstats.com/fight-details/031e6624f96a...,http://ufcstats.com/event-details/56116537d71a...,0,Mason Jones,Mike Davis,Lightweight,Orthodox,1995-04-26,U-DEC,...,NaN,NaN,0.133333,NaN,NaN,NaN,0.466667,NaN,NaN,NaN
11817,2021-01-20,http://ufcstats.com/fight-details/031e6624f96a...,http://ufcstats.com/event-details/56116537d71a...,1,Mike Davis,Mason Jones,Lightweight,Orthodox,1992-10-07,U-DEC,...,0.789963,NaN,0.609040,0.813559,0.609040,NaN,0.789454,1.050847,0.789454,NaN
11850,2021-01-23,http://ufcstats.com/fight-details/99dbe9669f31...,http://ufcstats.com/event-details/4304992c2acc...,0,Matt Frevola,Arman Tsarukyan,Lightweight,Orthodox,1990-06-11,U-DEC,...,0.355556,0.400000,0.093333,0.116667,0.155556,0.155556,0.106667,0.133333,0.177778,0.177778
11851,2021-01-23,http://ufcstats.com/fight-details/99dbe9669f31...,http://ufcstats.com/event-details/4304992c2acc...,1,Arman Tsarukyan,Matt Frevola,Lightweight,Orthodox,1996-10-11,U-DEC,...,0.511111,0.533333,0.550000,0.600000,0.733333,0.600000,0.766667,0.822222,1.022222,0.822222
11858,2021-01-23,http://ufcstats.com/fight-details/a5075f4eaee6...,http://ufcstats.com/event-details/4304992c2acc...,0,Dan Hooker,Michael Chandler,Lightweight,Switch,1990-02-13,KO/TKO,...,0.266667,0.266667,0.457739,0.488255,0.093333,0.582222,0.570447,0.608477,0.106667,0.662222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13291,2022-06-18,http://ufcstats.com/fight-details/4e6218b6b892...,http://ufcstats.com/event-details/a0a680fe2f6c...,1,Damir Ismagulov,Guram Kutateladze,Lightweight,Orthodox,1991-02-03,S-DEC,...,0.244444,0.066667,0.546667,0.683333,0.666667,0.666667,0.786667,0.983333,0.866667,0.866667
13314,2022-06-25,http://ufcstats.com/fight-details/029a12d36a46...,http://ufcstats.com/event-details/eb42d4febfaf...,0,Christos Giagos,Thiago Moises,Lightweight,Orthodox,1990-01-23,SUB,...,0.322170,0.344392,0.674544,0.741999,0.055710,0.344599,1.278332,1.406165,0.055710,0.411266
13315,2022-06-25,http://ufcstats.com/fight-details/029a12d36a46...,http://ufcstats.com/event-details/eb42d4febfaf...,1,Thiago Moises,Christos Giagos,Lightweight,Orthodox,1995-03-23,SUB,...,0.164819,0.056711,0.235931,0.265422,0.018904,0.018904,0.301491,0.339178,0.037807,0.037807
13320,2022-06-25,http://ufcstats.com/fight-details/1f5f59924b59...,http://ufcstats.com/event-details/eb42d4febfaf...,0,Arman Tsarukyan,Mateusz Gamrot,Lightweight,Orthodox,1996-10-11,U-DEC,...,0.080000,0.177778,2.190916,2.529402,4.378803,4.458803,2.772149,3.194174,5.446126,5.566126
